# Exercise 5 - Single Spike Train Analysis
Guy Singer
11.12.24

### Table of Contents

* [Part I: Spike Train Characteristics](#part1-spike-train)
   * [Last Week's Findings](#last-week)
   * [Interspike Interval Distribution](#interspike-interval)
   * [Poisson Spike Generator](#poisson-generator)
   * [Hazard and Survival Functions](#hazard-survival)

* [Part II: Cross- and Auto-correlation](#part2-correlation)
   * [Cross-correlation](#cross-correlation)
   * [Auto-correlation](#auto-correlation)
   * [Meaning of τ = 0 Value](#tau-zero)
   * [Predicting Neuron Type](#predict-neuron)
   * [Y-axis Values in Correlations](#y-axis)
   * [Non-stationarity](#non-stationarity)

## Part I: Spike Train Characteristics <a id='part1-spike-train'></a>

### Last Week's Findings <a id='last-week'></a>

Last week we deduced the expression for $P_T(n)$ or the model we use for the probability that any sequence of $n$ spikes occurs within a trial of duration $T$. After some inspection we came up with the following result:

$P_T(n) = \lim_{\Delta t \rightarrow 0} \binom{M}{n} (r \Delta t)^n (1 - r\Delta t)^{M - n}$

which can be simplified to

$P_T(n) = \frac{(rT)^n}{n!}e^{-rT}$

which also describes a Poisson process with a parameter of $rT$. From $P_T(n)$ we can deduce the chance for a *specific* spike train, i.e. a train with spikes at given times $\{t_1, \ldots, t_n\}$:

$P(t_1,\ldots,t_n) = n! P_T(n) \left( \frac{\Delta t}{T} \right)^n$

### Interspike Interval Distribution <a id='interspike-interval'></a>

A useful metric for spike trains in their interspike interval, i.e. the time between adjacent spikes. This statistical property is a way for us to discriminate between different neuronal behaviors in a quantitative manner. To arrive at that distribution, we assume that a spike occurred at time $t_i$ and we ask ourselves what is the probability (in an HPP) that another spike will occur in the short interval $t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t$ (figure 1, assuming $\Delta t$ is very small). That is, what is the chance for a spike $\tau$ seconds after the previous, with an interval of $\Delta t$ after it.

![figure](isi.png)
*Figure 1: Spike timing definitions.*

To answer, we need to see what is the probability that no spikes will fire for $\tau$ seconds, and then follow up with the probability that a spike will occur in the subsequent $\Delta t$ interval. The chance for $n=0$ spikes in the first interval is:

$P_\tau(n=0) = \frac{(r\tau)^0}{0!}e^{-r\tau} = e^{-r\tau}$

and the chance for a spike in a short interval $\Delta t$ is by definition $r \Delta t$. Thus, the probability of an interspike interval with a value of $\tau$ is:

$P(t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t) = r\Delta t e^{-rT}$

The PDF of interspike intervals is the same expression as above without the $\Delta t$, making it an exponential probability density function. From this inter-spike interval one can create the time interval histogram (TIH), which has an exponential trend, as seen in figure 2. These quantities, the ISI and the TIH indicate that long ISIs are generally less likely, at least in the case of the simple Poisson neuron. Any divergence from a simple exponential may indicate of some underlying property that will probably be of interest. For example, an oscillatory neuron will have a peak in its TIH at that expected time interval.

![figure](tih.png)
*Figure 2: Time interval histogram. X-axis is time in ms, Y-axis is percentage of total spikes. From Izhar Bar-Gad lectures.*

One notable difference we already see between a pure exponential and the TIH depicted above, is the number of ISIs in the lower-valued bins. Instead of the TIH obeying a simple exponential decay, it starts off very low, and only after about 10 ms it starts the expected decay process. This is due to the refractory period of real neurons, that was not accounted for during our derivation process. A more accurate function that describes the measured distribution is the Gamma function.

### Question
What is the expected mean interspike interval?

### Answer
We'll compute the expected value of the PDF to find the mean interval:

$\begin{aligned}
\left<\tau\right> & = \int_0^\infty \tau p(\tau) d\tau \\
& = \int_0^\infty \tau r e^{-r\tau} d\tau \\
& = r \int_0^\infty \tau e^{-r\tau} d \tau \text{\ \ \ (Gamma function)}\\
& = r \left[ \frac{1}{r^2} \right] = \frac{1}{r}
\end{aligned}$

This result is not surprising - for an average firing rate of $r$, the mean interval between spikes is $\frac{1}{r}$.

### Poisson Spike Generator <a id='poisson-generator'></a>

With the model of a Poisson process for a spike train, we can use $r(t)$ to simulate the firing of a neuron, by "abusing" the fact that the probability of firing a spike in a short interval of time $\Delta t$ is $r\Delta t$. To generate the spikes we create a vector with random numbers of the same length of our time series, and check whether the random number in that bin each greater than the $r(t)$ for that bin. If that is the case we determine that a spike occurred in that bin.

An important deviation of real neurons from a straight-forward Poisson process is their refractory period, as we mentioned during our discussion of the TIH. To account for it, we can very easily prohibit spikes in a time interval following a generated spikes. The resulting TIH from the generated data should closely resemble a "true" TIH as seen in figure 2.

Yet another real-world deviation from the Poisson process are bursting neurons. Burst activity is a very important feature of some neurons, and a simple spike generator fails to mimic it. Two general solution exist - either model the interval between bursts as a Poisson process, while modeling the spikes within the bursts themselves in a different manner, or increase the probability of firing after a spike by some factor and re-conduct the chance-to-spike calculation during that part.

### Hazard and Survival Functions <a id='hazard-survival'></a>

Integration of the probability density of the interspike intervals function yields probability. For example:

$P(\text{neuron fires until }t) = \int_{\hat{t}}^t r e^{-r(t'-\hat{t})}dt'$

where $\hat{t}$ is the time the neuron last fired. The probability the neuron *doesn't fire* in that period is called the *survival function*, and is just $S(t|\hat{t}) = 1 - P(\text{neuron fires until }t)$. 

The nickname was given to emphasize that the neuron has to "survive" until the end of the period without spiking. The initial value of the function is 1, and it decreases to 0 as $t \rightarrow \infty$. It can also be thought of as a 1 - (cumulitive sum of the time-interval histogram). The rate in which it decays is called the *hazard function*, and is written as:

$\rho(t|\hat{t}) = -\frac{d(S(t|\hat{t}))}{dt} \frac{1}{S(t|\hat{t})} = -\frac{P(\text{neuron fires until }t)}{S(t|\hat{t})}$

The hazard function can be thought of as the probability to spike at some given point given that it spiked before. After long ISIs it becomes noisy, since there are very few neurons that spike with such a long ISI, which influences the result hazard function value greatly.

## Part II: Cross- and Auto-correlation <a id='part2-correlation'></a>

### Cross-correlation <a id='cross-correlation'></a>

Given two signals, we can ask ourselves how similar they are. When calculating the correlation we look at the two signals in a specific time period or "window" and compare them. The mathematical definition is similar to convolution:

$(f \star g)(\tau) \equiv \sum_{n=-\infty}^\infty f[n] g[n+\tau]$

Intuitively, we go through every value in the system, multiplying both functions, after sliding one of the functions by a small step. In places where their values are both high throughout the spectrum, the multiplication will result in a higher value, creating a peak in the cross-correlation plot. Cross-correlation can identify time-shifting of functions easily, as for a certain value of $n$ we'll find a peak in the cross-correlation value.

![figure](xcorr.png)
*Figure 3: Signal of two sine waves (left) with their cross correlation. Notice the peak at $t=5$ time units.*

The demonstration shows two sine waves with the same frequency, one of them having been time-shifted by $\pi$ radians. The periodicity of 10 time units was preserved in the cross-correlation plot as well, and its peak at 5 time units indicates that one of the signals has been shifted by half a period from the other (hence the 5 time units difference). When dealing with neural data, the fact that cross-correlation histogram has a peak some distance away from the origin indicates a phase-locked response between both neurons.

### Question

Calculate the cross-correlation of the following two signals:

$f[n] = [-1 \  4 \ 2] \ ; \ g[n] = [8 \ -3 \ 4]$